In [17]:
# Define our machine

from collections import namedtuple, deque

MAIN_STACK_SIZE = 0x800
CALL_STACK_SIZE = 0x400

MAX_INT = 0x799F
MIN_INT = -0x8000

# Store the state of our machine
Machine = namedtuple(
    'Machine',
    ['main_stack',
     'call_stack',
     'instruction_pointer',
     'code'])


# Definition of instructions
Instruction = namedtuple(
    'Instruction',
    ['opcode',
     'name',
     'execute'])

def w(val):
    while val > MAX_INT:
        val -= 0x10000

    while val < MIN_INT:
        val += 0x10000

    return val


In [18]:
# Define some helpers

def push_stack(machine, val):
    if len(machine.main_stack) >= MAIN_STACK_SIZE:
        raise MachineError("Stack overflow: %x", machine)
    machine.main_stack.append(w(val))

    return machine


def pop_stack(machine):
    if len(machine.main_stack) == 0:
        raise MachineError("Stack underflow: %x", machine)

    return machine.main_stack.pop()


class MachineError(BaseException):
    def __init__(self, message, machine):
        super(MachineError, self).__init__(self, message)
        self.machine = machine


def make_machine(code=None):
    return Machine(
        main_stack=deque(), call_stack=deque(),
        instruction_pointer=0, code=code or []) 

In [19]:
# Define opcodes

# Stack operations
PUSH = 0x01
POP = 0x02
COPY = 0x03
SWAP = 0x04

# Arithmetic operations
ADD = 0x20
SUB = 0x21

# Control operations
JZ = 0x30
JG = 0x31
JL = 0x32
CALL = 0x33
RET = 0x34

# IO
PUTCH = 0x50
PUTDEC = 0x51
PUTHEX = 0x52
PUTS = 0x53

# Machine operations
HALT = 0x00

In [20]:
# 0x01 - PUSH
def _op_push(machine):
    # Get the next code word as the value to push
    word, machine = next_instruction(machine)
    return push_stack(machine, word)

# 0x03 - COPY
def _op_copy(machine):
    word = pop_stack(machine)
    return push_stack(push_stack(machine, word), word)


# 0x20 - ADD
def _op_add(machine):
    return push_stack(
        machine,
        pop_stack(machine) + pop_stack(machine))

# 0x21 - SUB
def _op_sub(machine):
    op2, op1 = (pop_stack(machine) for _ in range(2))
    return push_stack(machine, op1 - op2)

In [21]:
# 0x40 - JZ (Jump if zero)
def _op_jmp_zero(machine):
    new_ip, result = (pop_stack(machine) for _ in range(2))
    if result == 0:
        return Machine(
            machine.main_stack, machine.call_stack, new_ip, machine.code)
    return machine

# 0x41 - JG (Jump if greater)
def _op_jmp_greater(machine):
    new_ip, result = (pop_stack(machine) for _ in range(2))
    if result > 0:
        return Machine(
            machine.main_stack, machine.call_stack, new_ip, machine.code)
    return machine

# 0x42 - JG (Jump if greater)
def _op_jmp_less(machine):
    new_ip, result = (pop_stack(machine) for _ in range(2))
    if result < 0:
        return Machine(
            machine.main_stack, machine.call_stack, new_ip, machine.code)
    return machine

In [22]:
# 0x50 - PUTCH
def _op_putch(machine):
    val = pop_stack(machine)
    sys.stdout.write(chr(val))
    return machine

# 0x51 - PUTDEC
def _op_putdec(machine):
    val = pop_stack(machine)
    print(val)
    return machine

In [23]:
# A little helper for debugging instructions

def describe_instruction(instruction):
    return "0x%.2x -> %s" % instruction[0:2]

In [24]:
dispatch_table = {
    # Stack operations
    PUSH: Instruction(PUSH, "PUSH", _op_push),
    COPY: Instruction(COPY, "COPY", _op_copy),

    # Arithmetic operations
    ADD: Instruction(ADD, "ADD", _op_add),
    SUB: Instruction(SUB, "SUB", _op_sub),

    # Control operations
    JZ: Instruction(JZ, "JZ", _op_jmp_zero),
    JG: Instruction(JG, "JG", _op_jmp_greater),
    JL: Instruction(JL, "JL", _op_jmp_less),

    # IO operations
    PUTCH: Instruction(PUTCH, "PUTCH", _op_putch),
    PUTDEC: Instruction(PUTDEC, "PUTDEC", _op_putdec)
}

In [25]:
def next_instruction(machine):
    main_stack, call_stack, ip, code = machine

    # Bounds check on ip
    if ip >= len(code) or ip < 0:
        raise MachineError(
            "ip out of code range: ip=%d, code size=%d" % (ip, len(code)),
            machine)

    # Fetch next instruction opcode (or argument for certain opcodes)
    opcode = code[ip]

    # Return the machine with the instruction pointer incremented
    return opcode, Machine(main_stack, call_stack, ip+1, code)

In [26]:
def step_machine(machine, debug=True):
    opcode, machine = next_instruction(machine)

    # Match and execute on instruction with our dispatch table
    instruction = dispatch_table.get(opcode)
    if not instruction:
        raise MachineError("Got bad opcode: %x" % opcode, machine)
    if debug:
        print("++ Exec ip=%d [%s]" % (
            machine.instruction_pointer,
            describe_instruction(instruction)))

    return instruction.execute(machine)

In [27]:
def run_machine(machine, debug=True):
    """
    Run the machine until it errors or we hit a HALT instruction
    """

    while True:
        # Bounds check
        if len(machine.code) <= machine.instruction_pointer:
             raise MachineError(
                "ip out of code range: ip=%d, code size=%d" % (
                    machine.instruction_pointer,
                    len(machine.code)),
                machine)

        # Stop on HALT instruction
        if machine.code[machine.instruction_pointer] == HALT:
            if debug:
                print("++ HALT")
            return machine
        else:
            machine = step_machine(machine, debug)

In [28]:
def run_code_for_result(code, debug=False):
    """
    A shortcut for running some code and returning just the result
    """
    return run_machine(make_machine(code), debug).main_stack.pop()

In [29]:
PUSH_AND_HALT = [
    PUSH, 0x66, # PUSH 66
    HALT]       # HALT

run_code_for_result(PUSH_AND_HALT)
# Result -> 102 (66h)

102

In [30]:
ADD_TWO_AND_THREE = [
    PUSH, 0x2, # PUSH 2
    PUSH, 0x3, # PUSH 3
    ADD,       # ADD
    HALT]      # HALT

run_code_for_result(ADD_TWO_AND_THREE)
# Result -> 5

5

In [32]:
SUB_FIVE_FROM_TWO = [
    PUSH, 0x2, # PUSH 2
    PUSH, 0x5, # PUSH 5
    SUB,       # SUB
    HALT
]

run_code_for_result(SUB_FIVE_FROM_TWO)
# Result -> -3

-3

In [34]:
JMP_IF_ZERO = [
    PUSH, 0x0,
    PUSH, 0x8,
    JZ,

    PUSH, 0x100, # This should never run
    HALT,

    # This should always run
    PUSH, 0x200,
    HALT
]
# Result -> 0x200 (512h)
run_code_for_result(JMP_IF_ZERO)

512